In [4]:
from torch_contour.torch_contour import area, perimeter, hausdorff_distance
import torch

polygons1 = torch.tensor([
   [[0, 0], [1, 0], [1, 1], [0, 1]],  # Square
   [[0, 0], [2, 0], [2, 1], [0, 1]]   # Rectangle
], dtype=torch.float32).permute(0, 2, 1)  # Permute to shape (B, 2, N)
polygons2 = torch.tensor([
   [[0, 0], [1, 0], [1, 1], [0, 1]],  # Another Square
   [[0, 0], [2, 0], [2, 2], [0, 2]]   # Another Rectangle
], dtype=torch.float32).permute(0, 2, 1)  # Permute to shape (B, 2, N)


area_ = area(polygons1)
perimeter_ = perimeter(polygons2)
hausdorff_dists = hausdorff_distance(polygons1, polygons2)